# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1968dbbd30>
├── 'a' --> tensor([[-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233]])
└── 'x' --> <FastTreeValue 0x7f1968dbbc70>
    └── 'c' --> tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                        [-1.3001,  1.4418,  0.1090,  0.1279],
                        [-0.0194,  0.8076,  1.1247,  0.4720]])

In [4]:
t.a

tensor([[-0.0744, -1.6976,  0.3786],
        [-0.9852, -0.9755, -1.2233]])

In [5]:
%timeit t.a

65.9 ns ± 0.0385 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1968dbbd30>
├── 'a' --> tensor([[0.7460, 1.5393, 0.8395],
│                   [0.6709, 0.7732, 0.0985]])
└── 'x' --> <FastTreeValue 0x7f1968dbbc70>
    └── 'c' --> tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                        [-1.3001,  1.4418,  0.1090,  0.1279],
                        [-0.0194,  0.8076,  1.1247,  0.4720]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.146 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0744, -1.6976,  0.3786],
               [-0.9852, -0.9755, -1.2233]]),
    x: Batch(
           c: tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720]]),
       ),
)

In [10]:
b.a

tensor([[-0.0744, -1.6976,  0.3786],
        [-0.9852, -0.9755, -1.2233]])

In [11]:
%timeit b.a

71.8 ns ± 0.00697 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4631, -0.4416, -0.6703],
               [ 0.4748, -1.2254,  1.5674]]),
    x: Batch(
           c: tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720]]),
       ),
)

In [13]:
%timeit b.a = new_value

511 ns ± 0.289 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

812 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 608 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f18bba28130>
├── 'a' --> tensor([[[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]],
│           
│                   [[-0.0744, -1.6976,  0.3786],
│                    [-0.9852, -0.9755, -1.2233]]])
└── 'x' --> <FastTreeValue 0x7f18bba28fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 77.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1968d55370>
├── 'a' --> tensor([[-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233],
│                   [-0.0744, -1.6976,  0.3786],
│                   [-0.9852, -0.9755, -1.2233]])
└── 'x' --> <FastTreeValue 0x7f18bba0b0a0>
    └── 'c' --> tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                        [-1.3001,  1.4418,  0.1090,  0.1279],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 86.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2258, -1.2156, -0.7683, -0.2319],
                       [-1.3001,  1.4418,  0.1090,  0.1279],
                       [-0.0194,  0.8076,  1.1247,  0.4720]],
              
                      [[ 0.2258, -1.2156, -0.7683, -0.2319],
                       [-1.3001,  1.4418,  0.1090,  0.1279],
                       [-0.0194,  0.8076,  1.1247,  0.4720]],
              
                      [[ 0.2258, -1.2156, -0.7683, -0.2319],
                       [-1.3001,  1.4418,  0.1090,  0.1279],
                       [-0.0194,  0.8076,  1.1247,  0.4720]],
              
                      [[ 0.2258, -1.2156, -0.7683, -0.2319],
                       [-1.3001,  1.4418,  0.1090,  0.1279],
                       [-0.0194,  0.8076,  1.1247,  0.4720]],
              
                      [[ 0.2258, -1.2156, -0.7683, -0.2319],
                       [-1.3001,  1.4418,  0.1090,  0.1279],
                       [-0.0194,  0.8076,  1.1247,  0.4720]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 97.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720],
                      [ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720],
                      [ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720],
                      [ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720],
                      [ 0.2258, -1.2156, -0.7683, -0.2319],
                      [-1.3001,  1.4418,  0.1090,  0.1279],
                      [-0.0194,  0.8076,  1.1247,  0.4720],
                      [ 0.2258, -1.2156, -0.7683, -0.2319],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
